In [1]:
import json
import os

In [ ]:
import os

current = '/Users/jinym/Downloads/250923_중간납품_사전crop/'
# all = []
for root, dirs, files in os.walk(current):
    for file in files:
        os.system(f'unzip {os.path.join(root,file)} && mv crop /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/crops/{os.path.splitext(file)[0]}')
    #    all.append(os.path.join(root,file))

In [160]:
# rag_data = []
# for a in all:
#     if a.__contains__('rag'):
#         rag_data+=json.load(open(a, 'r'))

In [2]:
import pandas as pd
import os
import re
from collections import defaultdict

def extract_page_number(filename):
    """파일명에서 페이지 번호 추출"""
    # tb_0167_0001.png -> 0167, img_0158_0002.png -> 0158
    match = re.search(r'(?:tb_|img_)(\d{4})', filename)
    return match.group(1) if match else None

def organize_crop_files(crop_dir):
    """crop_dir의 폴더별로 파일들을 정리"""
    results = []
    
    for root, dirs, files in os.walk(crop_dir):
        # .DS_Store 파일 제외
        files = [f for f in files if f != '.DS_Store']
        
        if not files:  # 파일이 없으면 건너뛰기
            continue
            
        folder_name = os.path.basename(root)
        # if not folder_name:
            # continue
        
        # 파일들을 테이블과 이미지로 분류
        table_files = []
        image_files = []
        pages_with_files = set()
        
        for file in files:
            if file.startswith('tb_'):
                table_files.append(file)
                page_num = extract_page_number(file)
                if page_num:
                    pages_with_files.add(page_num)
            elif file.startswith('img_'):
                image_files.append(file)
                page_num = extract_page_number(file)
                if page_num:
                    pages_with_files.add(page_num)
        
        # 결과 저장
        results.append({
            '폴더명': folder_name,
            '테이블_파일_수': len(table_files),
            '테이블_파일_목록': ', '.join(sorted(table_files)),
            '이미지_파일_수': len(image_files),
            '이미지_파일_목록': ', '.join(sorted(image_files)),
            '파일이_있는_페이지_수': len(pages_with_files),
            '파일이_있는_페이지_목록': ', '.join(sorted(pages_with_files))
        })
    
    return results

# 실행
crop_dir = '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/crops/'
results = organize_crop_files(crop_dir)

# DataFrame 생성
df = pd.DataFrame(results)

# 결과 출력
print(f"총 {len(results)}개 폴더를 처리했습니다.")
print("\n처리 결과:")
print(df)

df.to_excel('crop_files_analysis_AFTER.xlsx', index=False)

총 214개 폴더를 처리했습니다.

처리 결과:
        폴더명  테이블_파일_수                                          테이블_파일_목록  \
0    SS0296        48  tb_0017_0001.png, tb_0018_0001.png, tb_0033_00...   
1    SS0291       152  tb_0017_0001.png, tb_0022_0001.png, tb_0024_00...   
2    SS0057        16  tb_0037_0001.png, tb_0037_0002.png, tb_0132_00...   
3    SS0265         5  tb_0297_0001.png, tb_0362_0001.png, tb_0367_00...   
4    SS0298        75  tb_0021_0001.png, tb_0056_0001.png, tb_0057_00...   
..      ...       ...                                                ...   
209  SS0022        45  tb_0032_0001.png, tb_0043_0001.png, tb_0051_00...   
210  SS0228        32  tb_0017_0001.png, tb_0028_0001.png, tb_0036_00...   
211  SS0217       137  tb_0017_0001.png, tb_0018_0001.png, tb_0018_00...   
212  SS0221         1                                   tb_0127_0001.png   
213  SS0226       264  tb_0009_0001.png, tb_0010_0001.png, tb_0010_00...   

     이미지_파일_수                                          이미지_파

In [3]:
df_before = pd.read_excel('crop_files_analysis.xlsx')

In [ ]:
set(df['폴더명']) - set(df_before['폴더명'])

In [ ]:
import pandas as pd
import re
from collections import defaultdict

def extract_page_numbers(file_list_str):
    """파일 목록 문자열에서 페이지 번호들을 추출"""
    if pd.isna(file_list_str) or file_list_str == '':
        return set()
    
    pages = set()
    files = file_list_str.split(', ')
    for file in files:
        # tb_0167_0001.png -> 0167, img_0158_0002.png -> 0158
        match = re.search(r'(?:tb_|img_)(\d{4})', file)
        if match:
            pages.add(match.group(1))
    return pages

def compare_analysis_files(before_file, after_file):
    """BEFORE와 AFTER 파일을 비교하여 차이점 분석"""
    
    # Excel 파일 읽기
    df_before = pd.read_excel(before_file)
    df_after = pd.read_excel(after_file)
    
    # 폴더명을 기준으로 매칭
    before_dict = {row['폴더명']: row for _, row in df_before.iterrows()}
    after_dict = {row['폴더명']: row for _, row in df_after.iterrows()}
    
    results = []
    
    for folder_name in before_dict.keys():
        if folder_name not in after_dict:
            # AFTER에 없는 폴더
            results.append({
                '폴더명': folder_name,
                '상태': 'AFTER에_없는_폴더',
                '테이블_파일_수': before_dict[folder_name]['테이블_파일_수'],
                '이미지_파일_수': before_dict[folder_name]['이미지_파일_수'],
                '없어진_테이블_파일': before_dict[folder_name]['테이블_파일_목록'],
                '없어진_이미지_파일': before_dict[folder_name]['이미지_파일_목록'],
                '없어진_페이지_수': before_dict[folder_name]['파일이_있는_페이지_수'],
                '없어진_페이지_목록': before_dict[folder_name]['파일이_있는_페이지_목록']
            })
        else:
            # 두 파일 모두에 있는 폴더 - 파일 비교
            before_row = before_dict[folder_name]
            after_row = after_dict[folder_name]
            
            # 파일 목록 비교
            before_table_files = set(before_row['테이블_파일_목록'].split(', ')) if pd.notna(before_row['테이블_파일_목록']) else set()
            after_table_files = set(after_row['테이블_파일_목록'].split(', ')) if pd.notna(after_row['테이블_파일_목록']) else set()
            
            before_image_files = set(before_row['이미지_파일_목록'].split(', ')) if pd.notna(before_row['이미지_파일_목록']) else set()
            after_image_files = set(after_row['이미지_파일_목록'].split(', ')) if pd.notna(after_row['이미지_파일_목록']) else set()
            
            # AFTER에 없는 파일들
            missing_table_files = before_table_files - after_table_files
            missing_image_files = before_image_files - after_image_files
            
            # 페이지 비교
            before_pages = set(before_row['파일이_있는_페이지_목록'].split(', '))
            after_pages = set(after_row['파일이_있는_페이지_목록'].split(', '))
            missing_pages = list(before_pages - after_pages)
            
            if missing_table_files or missing_image_files or missing_pages:
                results.append({
                    '폴더명': folder_name,
                    '상태': '일부_파일_누락',
                    '테이블_파일_수': before_row['테이블_파일_수'],
                    '이미지_파일_수': before_row['이미지_파일_수'],
                    '없어진_테이블_파일': ', '.join(sorted(missing_table_files)) if missing_table_files else '',
                    '없어진_이미지_파일': ', '.join(sorted(missing_image_files)) if missing_image_files else '',
                    '없어진_페이지_수': len(missing_pages),
                    '없어진_페이지_목록': ', '.join(sorted(missing_pages)) if missing_pages else '',
                    '최종_페이지_목록': ", ".join(sorted(after_pages))
                })
    
    return results

# 실행
before_file = 'crop_files_analysis.xlsx'
after_file = 'crop_files_analysis_AFTER.xlsx'

# 비교 분석 실행
comparison_results = compare_analysis_files(before_file, after_file)

# 결과를 DataFrame으로 변환
df_comparison = pd.DataFrame(comparison_results)

# 결과 출력
print(f"총 {len(comparison_results)}개 폴더에서 차이점을 발견했습니다.")
print("\n비교 결과:")
print(df_comparison)

# Excel 파일로 저장
df_comparison.to_excel('file_comparison_analysis.xlsx', index=False)

# 요약 통계
# total_missing_table_files = sum(len(row['없어진_테이블_파일'].split(', ')) if row['없어진_테이블_파일'] else 0 for row in comparison_results)
# total_missing_image_files = sum(len(row['없어진_이미지_파일'].split(', ')) if row['없어진_이미지_파일'] else 0 for row in comparison_results)
# total_missing_pages = sum(row['없어진_페이지_수'] for row in comparison_results)

# print(f"\n요약:")
# print(f"- 총 누락된 테이블 파일: {total_missing_table_files}개")
# print(f"- 총 누락된 이미지 파일: {total_missing_image_files}개")
# print(f"- 총 누락된 페이지: {total_missing_pages}개")